## Notes

This notebook was used to generate the test data for "CMatrix get w/ block small" in `test/units/contact_matrix/contact_matrix_test.cpp`

In [ ]:
import cooler
import numpy as np
import matplotlib.pyplot as plt
%load_ext cython

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

DTYPE = np.int32
ctypedef np.int32_t DTYPE_t

#@cython.boundscheck(False) # turn off bounds-checking for entire function
#@cython.wraparound(False)  # turn off negative index wrapping for entire function
def fetch_block_of_contacts_cython(np.ndarray[DTYPE_t, ndim=2] pixels, int row, int col, int bs = 9) -> int:
    cdef int first_row = row - (bs // 2)
    cdef int first_col = col - (bs // 2)
    cdef int n = 0
    cdef int nrows = pixels.shape[0]
    cdef int ncols = pixels.shape[1]
        
    cdef int i, j, ii, jj
    for i in range(first_row, first_row + bs):
        for j in range(first_col, first_col + bs):
            ii = min(max(0, i), nrows - 1)
            jj = min(max(0, j), ncols - 1)
            # print(f"i={i}; j={j}; ii={ii}; jj={jj};")
            n += pixels[ii, jj]
    #if n > 50000:
    #    print(i, j)
    return n


#@cython.boundscheck(False) # turn off bounds-checking for entire function
#@cython.wraparound(False)  # turn off negative index wrapping for entire function
def apply_fetch_block(np.ndarray[DTYPE_t, ndim=2] pixels, int bs = 9):
    cdef np.ndarray[DTYPE_t, ndim=2] out = np.empty_like(pixels, dtype=DTYPE)
    cdef int i, j
    cdef int nrows = pixels.shape[0]
    cdef int ncols = pixels.shape[1]
    

    for i in range(nrows):
        for j in range(ncols):
            out[i, j] = fetch_block_of_contacts_cython(pixels, i, j, bs)
    return out

In [ ]:
path_to_input_matrix = "/home/roby/github/modle_data_analysis/003_modle_pert_001/tmp_data/modle/modle_sim_hg38_reference_wo_noise.cool"
chrom_name = "chr1"
block_size = 9
base_output="/var/tmp/"

In [ ]:
c = cooler.Cooler(path_to_input_matrix)
chrom_idx = c.chromnames.index(chrom_name)
chrom_length = c.chromsizes[chrom_idx]
query = f"{chrom_name}:0-{chrom_length}"
input_pixels = np.array(c.matrix(balance=False).fetch(query), dtype=np.int32)
input_pixels

In [ ]:
output_pixels = apply_fetch_block(input_pixels, 9)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(2* 6.4, 6.4))
axs[0].imshow(input_pixels[:300,:300])
axs[1].imshow(output_pixels[:300,:300])

In [ ]:
#np.savetxt(f"{base_out}/contacts_{chrom_name}_raw.tsv.gz", contacts, delimiter="\t", fmt="%d")
#np.savetxt(f"{base_out}/contacts_{chrom_name}_bs9.tsv.gz", test, delimiter="\t", fmt="%d")

np.savetxt(f"{base_out}/contacts_{chrom_name}_raw_small.tsv", contacts[:81,:81], delimiter="\t", fmt="%d")
np.savetxt(f"{base_out}/contacts_{chrom_name}_bs9_small.tsv", apply_fetch_block(contacts[:81,:81], 9), delimiter="\t", fmt="%d")